# Import

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

from fairlearn.metrics import (demographic_parity_difference,
                               demographic_parity_ratio,
                               equalized_odds_difference,
                               equalized_odds_ratio,
                               MetricFrame,
                               false_positive_rate,
                               selection_rate,
                               true_positive_rate)

# Parameters

In [4]:
# parameters and configuration
INC_THRESHOLD = 50_000
ROUND_DIGITS = 4
SELECT_FEATURES = ['AGEP', 'COW', 'SCHL',
                   'MAR', 'OCCP', 'POBP',
                   'RELP', 'WKHP', 'SEX',
                   'RAC1P', 'ST']
SPLIT_SEED = 42
TEST_SIZE = 0.3

# Read data

In [2]:
df_1 = pd.read_csv("data/ACS_50k_RACE_3.csv", index_col="id")
df_2 = pd.read_csv("data/prompt 2 - 336 samples.csv", index_col="id")
df_4 = pd.read_csv("data/prompt 4 - 410 samples.csv", index_col="id")
df_5 = pd.read_csv("data/prompt 5 - 410 samples.csv", index_col="id")

df_og = pd.read_csv("data/ACS_50k.csv", index_col="id")
df_og_50k = df_og.copy()

# Pre-processing

In [3]:
dfs_synth = [df_1, df_2, df_4, df_5]

df_og = df_og[df_og.RAC1P == 9]

for idx, df in enumerate(dfs_synth):
    dfs_synth[idx] = pd.concat([df, df_og], axis=0, ignore_index=True)
    dfs_synth[idx].reset_index()

# Data splitting

In [5]:
def split_data(df):

    X = df[SELECT_FEATURES]
    # df['INC_BINARY'] = (df['PINCP'] > INC_TRESHOLD).astype(int)
    y = df['PINCP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=SPLIT_SEED)
    y_test = (y_test > INC_THRESHOLD).astype(int)

    return X_train, X_test, y_train, y_test

# Training

In [9]:
def train_models(X_train, y_train):
    ml_algos = [DecisionTreeRegressor, LinearRegression]

    models_dict = {}
    for algo in ml_algos:
        model = algo()
        model.fit(X_train, y_train)
        models_dict[algo.__name__] = model
    return models_dict

# Metrics

In [58]:
def _make_predictions(model_dict, X_test):
    preds_dict = {}
    for algo, model in model_dict.items():
        pred = model.predict(X_test)
        pred = (pred > INC_THRESHOLD).astype(int)
        preds_dict[algo] = pred
    return preds_dict


def get_ml_metrics(model_dict, X_test, y_test):
    preds_dict = _make_predictions(model_dict, X_test)
    metrics_dict = {}
    for algo, pred in preds_dict.items():
        metrics_dict[algo] = {}
        metrics_dict[algo]['Accuracy'] = round(accuracy_score(y_test, pred), ndigits=ROUND_DIGITS)
        metrics_dict[algo]['F1'] = round(f1_score(y_test, pred), ndigits=ROUND_DIGITS)
    return metrics_dict


def _get_parity_frame(model_dict, y_true, X_test):
        preds_dict = _make_predictions(model_dict, X_test)
        frames_dict = {}
        for algo, pred in preds_dict.items():
            sel_rate = MetricFrame(
                metrics=selection_rate,
                y_true=y_true,
                y_pred=pred,
                sensitive_features=X_test['RAC1P']
            )
            frames_dict[algo] = sel_rate
        return frames_dict


def _get_equalized_odds_frame(model_dict, y_true, X_test):
        preds_dict = _make_predictions(model_dict, X_test)
        frames_dict = {}
        for algo, pred in preds_dict.items():
            fns = {"tpr": true_positive_rate, "fpr": false_positive_rate}
            eo = MetricFrame(
                metrics=fns,
                y_true=y_true,
                y_pred=pred,
                sensitive_features=X_test['RAC1P'],
            )
            frames_dict[algo] = eo
        return frames_dict


def get_fairness_frames(model_dict, y_true, X_test):
    frames_dict = {}
    for algo, model in model_dict.items():
          frames_dict[algo] = {}
          parity_dict = _get_parity_frame({algo: model}, y_true, X_test)
          eo_dict = _get_equalized_odds_frame({algo: model}, y_true, X_test)
          frames_dict[algo]['Demographic parity frame'] = parity_dict[algo]
          frames_dict[algo]['Equalized odds frame'] = eo_dict[algo]
    return frames_dict
     


def get_fairness_metrics(model_dict, X_test, y_test):
    preds_dict = _make_predictions(model_dict, X_test)
    metrics_dict = {}
    for algo, pred in preds_dict.items():
        metrics_dict[algo] = {}
        metrics_dict[algo]['Demographic parity difference'] = \
            demographic_parity_difference(y_test, pred, sensitive_features=X_test['RAC1P'])
        metrics_dict[algo]['Demographic parity ratio'] = \
            demographic_parity_ratio(y_test, pred, sensitive_features=X_test['RAC1P'])
        metrics_dict[algo]['Equalized odds difference'] = \
            equalized_odds_difference(y_test, pred, sensitive_features=X_test['RAC1P'])
        metrics_dict[algo]['Equalized odds ratio'] = \
            equalized_odds_ratio(y_test, pred, sensitive_features=X_test['RAC1P'])
    return metrics_dict

# Main code

First, analyzing the whole 50k dataset by running fairness metrics on all races

Second, analyzing the merged synthetic datasets

In [60]:
# Analyze fairness metrics on whole 50k dataset

X_train, X_test, y_train, y_test = split_data(df_og_50k)
model_dict = train_models(X_train, y_train)
fairness_metrics = get_fairness_metrics(model_dict, X_test, y_test)
frames = get_fairness_frames(model_dict, y_test, X_test)
for algo, frame_dict in frames.items():
    print(algo)
    print()
    for name, frame in frame_dict.items():
        print(name)
        display(frame.by_group)

DecisionTreeRegressor

Demographic parity frame


RAC1P
1.0    0.394726
2.0    0.276580
3.0    0.211009
4.0    0.250000
5.0    0.166667
6.0    0.464411
7.0    0.193548
8.0    0.207641
9.0    0.357746
Name: selection_rate, dtype: float64

Equalized odds frame


,tpr,fpr
RAC1P,,
1.0,0.633731,0.233997
2.0,0.535211,0.207352
3.0,0.500000,0.111111
4.0,0.000000,0.333333
5.0,0.500000,0.000000
6.0,0.686016,0.288703
7.0,0.375000,0.130435
8.0,0.510204,0.148810
9.0,0.612613,0.241803


LinearRegression

Demographic parity frame


RAC1P
1.0    0.611953
2.0    0.484015
3.0    0.440367
4.0    0.250000
5.0    0.500000
6.0    0.661610
7.0    0.580645
8.0    0.315615
9.0    0.478873
Name: selection_rate, dtype: float64

Equalized odds frame


,tpr,fpr
RAC1P,,
1.0,0.895017,0.421595
2.0,0.845070,0.387370
3.0,0.785714,0.320988
4.0,0.000000,0.333333
5.0,1.000000,0.250000
6.0,0.918206,0.458159
7.0,0.875000,0.478261
8.0,0.734694,0.234127
9.0,0.828829,0.319672


In [61]:
# Analyze the 4 merged datasets

import pprint

for idx, df in enumerate(dfs_synth):
    X_train, X_test, y_train, y_test = split_data(df)
    model_dict = train_models(X_train, y_train)
    ml_metrics = get_ml_metrics(model_dict, X_test, y_test)
    fairness_metrics = get_fairness_metrics(model_dict, X_test, y_test)

    print("Dataframe", idx+1)
    print("Class balance:")
    pprint.pprint(y_test.value_counts().to_dict())
    pprint.pprint(ml_metrics)
    pprint.pprint(fairness_metrics)
    print()

Dataframe 1
Class balance:
{0: 342, 1: 140}
{'DecisionTreeRegressor': {'Accuracy': 0.7303, 'F1': 0.5548},
 'LinearRegression': {'Accuracy': 0.7635, 'F1': 0.678}}
{'DecisionTreeRegressor': {'Demographic parity difference': 0.02140264606018033,
                           'Demographic parity ratio': 0.9332310614361896,
                           'Equalized odds difference': 0.02851524090462143,
                           'Equalized odds ratio': 0.9107142857142858},
 'LinearRegression': {'Demographic parity difference': 0.1461187214611872,
                      'Demographic parity ratio': 0.6952380952380952,
                      'Equalized odds difference': 0.2359882005899705,
                      'Equalized odds ratio': 0.738562091503268}}

Dataframe 2
Class balance:
{0: 337, 1: 134}
{'DecisionTreeRegressor': {'Accuracy': 0.7771, 'F1': 0.6263},
 'LinearRegression': {'Accuracy': 0.7261, 'F1': 0.6195}}
{'DecisionTreeRegressor': {'Demographic parity difference': 0.14471721922679823,
      